# SFTTrainerを使用した教師あり微調整

このノートブックでは、`trl`ライブラリの`SFTTrainer`を使用して`HuggingFaceTB/SmolLM2-135M`モデルを微調整する方法を示します。ノートブックのセルを実行すると、モデルの微調整が行われます。さまざまなデータセットを試して、難易度を選択できます。

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>演習: SFTTrainerを使用したSmolLM2の微調整</h2>
    <p>Hugging Faceのリポジトリからデータセットを取得し、それを使用してモデルを微調整します。</p>
    <p><b>難易度レベル</b></p>
    <p>🐢 `HuggingFaceTB/smoltalk`データセットを使用</p>
    <p>🐕 `bigcode/the-stack-smol`データセットを試し、特定のサブセット`data/python`でコード生成モデルを微調整します。</p>
    <p>🦁 実際の使用ケースに関連するデータセットを選択します。</p>
</div>

In [ ]:
# Google Colabでの要件のインストール
# !pip install transformers datasets trl huggingface_hub


# Hugging Faceへの認証
from huggingface_hub import login

login()

# 便利のため、Hugging Faceのトークンを.envファイルのHF_TOKENとして環境変数に設定できます

In [ ]:
# 必要なライブラリをインポート
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

# デバイスを動的に設定
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# モデルとトークナイザーをロード
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# チャット形式を設定
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# 微調整されたモデルを保存および/またはアップロードするための名前を設定
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

# ベースモデルでの生成

ここでは、チャットテンプレートを持たないベースモデルを試してみます。

In [ ]:
# トレーニング前のベースモデルを試してみましょう
prompt = "プログラミングについての俳句を書いてください"

# テンプレートでフォーマット
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# 応答を生成
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("トレーニング前:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

## データセットの準備

例としてデータセットをロードし、トレーニング用にフォーマットします。データセットは、各入力がプロンプトで、出力がモデルの期待される応答である入力-出力ペアで構成されている必要があります。

**TRLは、モデルのチャットテンプレートに従って入力メッセージをフォーマットします。** これらは、`role`と`content`キーを持つ辞書のリストとして表現される必要があります。


In [ ]:
# 例としてデータセットをロード
from datasets import load_dataset

# TODO: パスと名前のパラメータを使用してデータセットと設定を定義
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

In [ ]:
# TODO: 🦁 データセットがTRLでチャットテンプレートに変換できない形式の場合、処理が必要です。 [モジュール](../chat_templates.md)を参照してください。

## SFTTrainerの設定

`SFTTrainer`は、トレーニングプロセスを制御するさまざまなパラメータで設定されます。これには、トレーニングステップの数、バッチサイズ、学習率、および評価戦略が含まれます。これらのパラメータを特定の要件と計算リソースに応じて調整します。

In [ ]:
# SFTTrainerを設定
sft_config = SFTConfig(
    output_dir="./sft_output",  # トレーニング結果の出力ディレクトリ
    max_steps=1000,  # データセットのサイズとトレーニングの所要時間に応じて調整
    per_device_train_batch_size=4,  # GPUのメモリ容量に応じて調整
    learning_rate=5e-5,  # 微調整の一般的な開始点
    logging_steps=10,  # トレーニングメトリックのログ記録頻度
    save_steps=100,  # モデルのチェックポイント保存頻度
    evaluation_strategy="steps",  # 定期的な間隔でモデルを評��
    eval_steps=50,  # 評価の頻度
    use_mps_device=(
        True if device == "mps" else False
    ),  # 混合精度トレーニングにMPSを使用
    hub_model_id=finetune_name,  # モデルに一意の名前を付ける
)

# SFTTrainerを初期化
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)

# TODO: 🦁 🐕 選択したデータセットに合わせてSFTTrainerのパラメータを調整します。例えば、`bigcode/the-stack-smol`データセットを使用する場合、`content`列を選択する必要があります。


## モデルのトレーニング

トレーナーが設定されたので、モデルのトレーニングを進めることができます。トレーニングプロセスは、データセットを反復し、損失を計算し、この損失を最小化するためにモデルのパラメータを更新することを含みます。

In [ ]:
# モデルをトレーニング
trainer.train()

# モデルを保存
trainer.save_model(f"./{finetune_name}")

In [ ]:
trainer.push_to_hub(tags=finetune_tags)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>追加の演習: 微調整されたモデルでの生成</h2>
    <p>🐕 微調整されたモデルを使用して、ベースモデルの例と同じように応答を生成します。</p>
</div>


In [ ]:
# 同じメッセージで微調整されたモデルをテスト

# トレーニング前のベースモデルを試してみましょう
prompt = "プログラミングについての俳句を書いてください"

# テンプレートでフォーマット
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# 応答を生成
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: 微調整されたモデルを使用して、ベースモデルの例と同じように応答を生成します。

## 💐 完了しました！

このノートブックでは、`SFTTrainer`を使用して`HuggingFaceTB/SmolLM2-135M`モデルを微調整するためのステップバイステップガイドを提供しました。これらの手順に従うことで、特定のタスクをより効果的に実行するためにモデルを適応させることができます。このコースを続けて作業したい場合、以下の提案を試してみてください：

- より高い難易度レベルでこのノートブックを試してみてください。
- 同僚のPRをレビューしてください。
- IssueやPRを通じてコースの資料を改善してください。
